In [1]:
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup as bs
from io import StringIO


# Início da Coleta Real de Dados

In [2]:
#years = list(range(2018,2025))
years = list(range(2024,2019,-1))

In [3]:
years

[2024, 2023, 2022, 2021, 2020]

In [4]:
all_matches = []

In [5]:
standings_url = "https://fbref.com/pt/comps/32/{ano}-{ano1}/{ano}-{ano1}-Primeira-Liga-estatisticas"

for year in years:
    data = requests.get(standings_url.replace("{ano}", str(year-1)).replace("{ano1}", str(year)))
    if data.status_code != 200:
        print(f'{year-1}-{year} - Temporada Não Disponível')
        continue;
    soup = bs(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get('href') for l in standings_table.find_all('a')]
    links = [l for l in links if '/equipes/' in l]
    team_urls = [f'https://fbref.com{l}' for l in links]

    # previous_season = soup.select('a.prev')[0].get('href')
    # standings_url = f"https://fbref.com/{previous_season}"

    for team_url in team_urls:
        team_name = team_url.split('/')[-1].replace('-Estatisticas', '').replace('-', ' ')

        data = requests.get(team_url)
        matches = pd.read_html(StringIO(data.text), match='Resultados e Calendários')[0]

        soup = bs(data.text)
        links = [l.get('href') for l in soup.find_all('a')]
        links = [l for l in links if l and '/partidas/all_comps/shooting/' in l]
        print(f'https://fbref.com{links[0]}')
        data = requests.get(f'https://fbref.com{links[0]}')
        shooting = pd.read_html(StringIO(data.text), match='Chutes')[0]
        shooting.columns = shooting.columns.droplevel()

        try:
            team_data = matches.merge(shooting[['Data', 'TC', 'CaG', 'Dist', 'G/Sh', 'FK', 'PB', 'PT']], on='Data')
        except ValueError:
            continue

        team_data = team_data[team_data['Camp.'] == 'Primeira Liga']
        team_data['Temporada'] = f'{year}-{year+1}'
        team_data['Time'] = team_name
        all_matches.append(team_data)
        time.sleep(6)

https://fbref.com/pt/equipes/a77c513e/2023-2024/partidas/all_comps/shooting/Benfica-Historicos-dos-Jogos-Todos-os-campeonatos
https://fbref.com/pt/equipes/13dc44fd/2023-2024/partidas/all_comps/shooting/Sporting-CP-Historicos-dos-Jogos-Todos-os-campeonatos
https://fbref.com/pt/equipes/5e876ee6/2023-2024/partidas/all_comps/shooting/Porto-Historicos-dos-Jogos-Todos-os-campeonatos
https://fbref.com/pt/equipes/69d84c29/2023-2024/partidas/all_comps/shooting/Braga-Historicos-dos-Jogos-Todos-os-campeonatos
https://fbref.com/pt/equipes/3f319bc9/2023-2024/partidas/all_comps/shooting/Vitoria-Guimaraes-Historicos-dos-Jogos-Todos-os-campeonatos
https://fbref.com/pt/equipes/e4502862/2023-2024/partidas/all_comps/shooting/Moreirense-Historicos-dos-Jogos-Todos-os-campeonatos
https://fbref.com/pt/equipes/489c9cd9/2023-2024/partidas/all_comps/shooting/Farense-Historicos-dos-Jogos-Todos-os-campeonatos
https://fbref.com/pt/equipes/0d36ddd4/2023-2024/partidas/all_comps/shooting/Arouca-Historicos-dos-Jogos-T

In [ ]:
match_df = pd.concat(all_matches, ignore_index=True)

In [ ]:
match_df['Camp'] = match_df['Camp.']
match_df = match_df.drop('Camp.', axis='columns')

In [ ]:
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df.to_csv('portugal_matches.csv')